In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
#checking if ready 

In [102]:
es=Elasticsearch([{"host":"localhost","port":9200}],http_auth=("elastic","suVlWmYRKKFk6RYs_TrU"))
print(es.ping())

True


In [103]:
df=pd.read_csv('buildler2.csv')

In [104]:
df = df.where(pd.notnull(df), None)

In [105]:
path_of_exile_builds_index = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "custom_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "asciifolding",
                            "custom_edge_ngram"
                        ]
                    }
                },
                "filter": {
                    "custom_edge_ngram": {
                        "type": "edge_ngram",
                        "min_gram": 2,
                        "max_gram": 10
                    }
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "Build": {
                "type": "text",
                "analyzer": "custom_analyzer"
            },
            "Ascendancy": {
                "type": "keyword"
            },
            "Açıklama": {
                "type": "text",
                "analyzer": "custom_analyzer"
            }
        }
    }
}

In [106]:
def dataframe_to_es(df, es_index):
    for df_idx, line in df.iterrows():
        yield {
            "_index": es_index,
            "_id": df_idx,  # Satır numarasını Elasticsearch belgesi ID'si olarak kullanıyoruz
            "_source": {
                "Build": line['Build'],
                "Ascendancy": line['Ascendancy'],
                "Açıklama": line['Açıklama']
            }
        }

In [107]:
try:
    es.indices.delete("path_of_exile_builds_index")
except:
    print("No index")

In [108]:
es.indices.create(index="path_of_exile_builds_index", body=path_of_exile_builds_index)  # <--- Eklenen satır

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'path_of_exile_builds_index'}

In [109]:
bulk(es, dataframe_to_es(df, "path_of_exile_builds_index"), raise_on_error=False)

(33, [])

In [114]:
search_query = " aura " #ENTER THE SEARCH INPUT HERE

response = es.search(index='path_of_exile_builds_index', body={
    "query": {
        "match": {
            "Build": {
                "query": search_query,
                "operator": "and"  # <--- Eklenen satır
            }
        }
    }
})

In [115]:
for hit in response['hits']['hits']:
    print(hit['_source'])

{'Build': 'aurabot', 'Ascendancy': None, 'Açıklama': 'Aurabot; müttefiklere çeşitli destek auraları sağlayan bir build.'}
